In [24]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report

# Config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 7
CLASS_NAMES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
CLASS_TO_IDX = {label: idx for idx, label in enumerate(CLASS_NAMES)}

# Dataset (landmarks only)
class LandmarkOnlyDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        landmark_path = row['Landmark']
        landmarks = np.load(landmark_path).astype(np.float32).flatten()  # (468, 2) → (936,)
        landmarks = torch.tensor(landmarks, dtype=torch.float32)
        label = CLASS_TO_IDX[row['Emotion']]
        return landmarks, label

# Loaders
def create_loaders(train_csv, test_csv):
    train_dataset = LandmarkOnlyDataset(train_csv)
    test_dataset = LandmarkOnlyDataset(test_csv)

    # Calculate class weights
    labels = [label for _, label in train_dataset]
    counts = np.bincount(labels, minlength=NUM_CLASSES)
    total = float(sum(counts))
    weights = [total / (NUM_CLASSES * count) for count in counts]
    class_weights = torch.tensor(weights, dtype=torch.float32)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    return train_loader, test_loader, class_weights

# Simple MLP Model
class LandmarkClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1404, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, NUM_CLASSES)
        )

    def forward(self, landmarks):
        return self.net(landmarks)


# Evaluation
def evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for landmarks, labels in dataloader:
            landmarks = landmarks.to(device)
            outputs = model(landmarks)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())
    report = classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=2)
    print("\nClassification Report:\n", report)
    return np.mean(np.array(y_true) == np.array(y_pred)) * 100

# Training
def train_model(train_csv, test_csv, num_epochs=25):
    train_loader, test_loader, class_weights = create_loaders(train_csv, test_csv)
    model = LandmarkClassifier().to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for landmarks, labels in pbar:
            landmarks, labels = landmarks.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(landmarks)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            pbar.set_postfix(loss=running_loss / (total // 32 + 1), acc=100 * correct / total)

        acc = evaluate(model, test_loader)
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_landmark_model.pth")
        print(f"Epoch {epoch+1} complete. Test Accuracy: {acc:.2f}%\n")

# Run
if __name__ == "__main__":
    train_csv = r"C:/Users/koushika/Dropbox/PC/Downloads/facemesh_data_train_landmarks.csv"
    test_csv = r"C:/Users/koushika/Dropbox/PC/Downloads/facemesh_data_test_landmarks.csv"
    train_model(train_csv, test_csv, num_epochs=25)


Epoch 1/25: 100%|██████████| 830/830 [00:15<00:00, 52.63it/s, acc=18.5, loss=1.94]
c:\Users\koushika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\koushika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\koushika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predic


Classification Report:
               precision    recall  f1-score   support

       angry       0.00      0.00      0.00       835
     disgust       0.00      0.00      0.00        97
        fear       0.00      0.00      0.00       933
       happy       0.53      0.01      0.01      1713
     neutral       0.23      0.25      0.24      1181
         sad       0.20      0.47      0.28      1094
    surprise       0.17      0.60      0.27       779

    accuracy                           0.19      6632
   macro avg       0.16      0.19      0.11      6632
weighted avg       0.23      0.19      0.12      6632

Epoch 1 complete. Test Accuracy: 19.41%



Epoch 2/25: 100%|██████████| 830/830 [00:21<00:00, 39.41it/s, acc=22, loss=1.92]  



Classification Report:
               precision    recall  f1-score   support

       angry       0.14      0.03      0.05       835
     disgust       0.00      0.00      0.00        97
        fear       0.14      0.09      0.11       933
       happy       0.45      0.52      0.48      1713
     neutral       0.19      0.00      0.01      1181
         sad       0.25      0.32      0.28      1094
    surprise       0.19      0.61      0.29       779

    accuracy                           0.28      6632
   macro avg       0.20      0.23      0.17      6632
weighted avg       0.25      0.28      0.23      6632

Epoch 2 complete. Test Accuracy: 27.61%



Epoch 3/25: 100%|██████████| 830/830 [00:22<00:00, 36.85it/s, acc=27.1, loss=1.88]


...


Classification Report:
               precision    recall  f1-score   support

       angry       0.41      0.12      0.18       835
     disgust       0.08      0.20      0.12        97
        fear       0.28      0.01      0.01       933
       happy       0.75      0.75      0.75      1713
     neutral       0.34      0.74      0.47      1181
         sad       0.36      0.30      0.33      1094
    surprise       0.54      0.67      0.60       779

    accuracy                           0.47      6632
   macro avg       0.40      0.40      0.35      6632
weighted avg       0.47      0.47      0.43      6632

Epoch 23 complete. Test Accuracy: 47.38%



Epoch 24/25: 100%|██████████| 830/830 [00:24<00:00, 33.59it/s, acc=43.3, loss=1.58]



Classification Report:
               precision    recall  f1-score   support

       angry       0.32      0.28      0.30       835
     disgust       0.06      0.53      0.10        97
        fear       0.21      0.02      0.03       933
       happy       0.76      0.76      0.76      1713
     neutral       0.39      0.47      0.43      1181
         sad       0.35      0.31      0.33      1094
    surprise       0.59      0.63      0.61       779

    accuracy                           0.45      6632
   macro avg       0.38      0.43      0.37      6632
weighted avg       0.46      0.45      0.44      6632

Epoch 24 complete. Test Accuracy: 45.13%



Epoch 25/25: 100%|██████████| 830/830 [00:24<00:00, 34.10it/s, acc=43.4, loss=1.57]



Classification Report:
               precision    recall  f1-score   support

       angry       0.35      0.21      0.26       835
     disgust       0.06      0.44      0.11        97
        fear       0.24      0.01      0.02       933
       happy       0.76      0.77      0.76      1713
     neutral       0.38      0.57      0.45      1181
         sad       0.36      0.29      0.32      1094
    surprise       0.53      0.70      0.60       779

    accuracy                           0.46      6632
   macro avg       0.38      0.43      0.36      6632
weighted avg       0.46      0.46      0.44      6632

Epoch 25 complete. Test Accuracy: 46.32%



In [25]:
import pandas as pd

# Update the path to your CSV file
csv_path = r"C:/Users/koushika/Dropbox/PC/Downloads/facemesh_data_train_landmarks.csv"

# Load and inspect
df = pd.read_csv(csv_path)

# Print the first few rows and the headers
print("CSV Headers:\n", df.columns.tolist())
print("\nFirst 5 Rows:\n", df.head())


CSV Headers:
 ['Image', 'Landmark', 'Emotion']

First 5 Rows:
                    Image                                           Landmark  \
0  Training_10118481.jpg  C:/Users/koushika/Dropbox/PC/Downloads/landmar...   
1  Training_10120469.jpg  C:/Users/koushika/Dropbox/PC/Downloads/landmar...   
2  Training_10161559.jpg  C:/Users/koushika/Dropbox/PC/Downloads/landmar...   
3   Training_1021836.jpg  C:/Users/koushika/Dropbox/PC/Downloads/landmar...   
4  Training_10269675.jpg  C:/Users/koushika/Dropbox/PC/Downloads/landmar...   

  Emotion  
0   angry  
1   angry  
2   angry  
3   angry  
4   angry  
